In [ ]:
!pip install qdrant-client openai dotenv langchain-openai pydantic

In [ ]:
from qdrant_client import QdrantClient
from openai import OpenAI
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import os

# ==== Configuration ====
#TODO: replace with your actual keys
QDRANT_URL = "your_qdrant_url"       # e.g. "https://xxxx-xxxxx.eu-central.aws.cloud.qdrant.io"
QDRANT_API_KEY = "your_qdrant_api_key"     # from your Qdrant Cloud dashboard
OPENAI_API_KEY = "your_openai_api_key"
EMBED_MODEL = "text-embedding-3-small"  # Must match the one used when uploading
# ========================

# === Initialize clients ===
client = OpenAI()
llm = ChatOpenAI(
    temperature=0,
    model_name="gpt-4.1-mini",
)

qdrant = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

from openai import OpenAI

client = OpenAI()


#TODO: add more channels if needed
collection_to_language = {
    "bluepigeon0810_videos": "Traditional Chinese",
}


def translate_query_if_needed(query: str, target_lang="English") -> str:
    """
    Use an LLM to detect the query language and translate to English only if needed.
    Keeps cost low by combining detection + translation in one step.
    """
    detection_prompt = f"""
    Detect the language of the following text. 
    If it's already in {target_lang}, reply with "{target_lang}".
    Otherwise, translate it into {target_lang} and output only the translation.

    Text: {query}
    """

    detection = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": "You are a precise language detector and translator."},
            {"role": "user", "content": detection_prompt}
        ]
    )

    result = detection.choices[0].message.content.strip()

    if result.upper() == target_lang.upper():
        print(f"🌐 Query is already in {target_lang} — skipping translation.")
        return query
    else:
        print(f"🌐 Translated query: {result}")
        return result

  


# === Define search function ===
def search_videos(collection_name: str, query: str, top_k: int = 3):
    """Embed the query and search the Qdrant vector store."""
    # 1️⃣ Get embedding for the query
    query = translate_query_if_needed(query, target_lang=collection_to_language.get(collection_name, "English"))
    emb = client.embeddings.create(
        model=EMBED_MODEL,
        input=query
    ).data[0].embedding

    # 2️⃣ Perform search in Qdrant
    response = qdrant.query_points(
        collection_name=collection_name,
        query=emb,
        limit=top_k,
        with_payload=True,  
        with_vectors=False   
    )
    return response.points
    # return results


# === Define structured response schema ===
class QueryResponse(BaseModel):
    can_be_answered: bool = Field(
        ..., description="Whether the question can be answered based on the video transcript."
    )
    summary: str = Field(
        ..., description="A concise summary answer if it can be answered, otherwise an empty string."
    )
    start_time: str = Field(
        ..., description="The start time (HH:MM:SS) where the answer appears, or empty string if not applicable."
    )
    end_time: str = Field(
        ..., description="The end time (HH:MM:SS) where the answer appears, or empty string if not applicable."
    )

class FinalSummary(BaseModel):
    can_be_answered: bool = Field(
        ..., description="Whether the question can be answered based on the combined video analyses."
    )
    final_answer: str = Field(
        ..., description="The final answer derived from the video analyses."
    )
    sources: list[str] = Field(
        ..., description="List of source video IDs that contributed to the final answer."
    )
    time_ranges: list[str] = Field(
        ..., description="List of source time ranges (HH:MM:SS - HH:MM:SS) corresponding to each source video."
    )

# === Define system prompt for the agent ===

SYSTEM_PROMPT = [
    "You are an intelligent video transcript analyst.",
    "Given:",
    "- A video transcript",
    "- The upload date of the video",
    "- A user question",
    "",
    "Your job is to:",
    "1. Determine if the question can be answered from the transcript.",
    "2. If yes:",
    "   - Provide a clear and detailed summary answer.",
    "   - Estimate the start and end times (HH:MM:SS format) where the relevant content appears, based on transcript clues.",
    "3. If not, respond with can_be_answered = false and leave other fields empty.",
    "",
    "Temporal reasoning rules:",
    "- Always consider both the **upload date** and the **time period described** within the transcript.",
    # "- If the transcript clearly refers to a specific time period (e.g., “in 2018”, “during the pandemic”, “earlier that year”), treat the statements as referring to that period — not necessarily the upload date.",
    # "- If the user’s question asks about a specific year or period, only include information that explicitly refers to or accurately describes that same time frame.",  
    "- Be objective and factual — do not guess or infer information beyond what the transcript supports.",
    "- The output language **must match the user question’s language** exactly.",
]
SYSTEM_PROMPT = "\n".join(SYSTEM_PROMPT)


JUDGE_PROMPT = [
    "You are an expert fact-checker and video analyst.",
    "",
    "**Important:** You may ONLY use the information explicitly provided in or inferred from the given video transcript analyses, summaries, time ranges, and upload dates. ",
    "Do NOT use any external knowledge, assumptions, or prior information about the topic.",
    "",
    "Given multiple video transcript analyses (with summaries, time ranges, and upload dates), produce a **concise, user-facing answer**:",
    "",
    "1. **Conciseness**",
    "   - Provide a short, direct answer to the user question.",
    "   - Do NOT list all video IDs, titles, or detailed reasoning in the final answer.",
    "   - Only include sources internally for validation or logging if needed.",    
    "",
    "2. **Time awareness**",
    "   - Take into account the time period being asked about.",
    "   - Prefer newer and more accurate videos if conflicts exist.",
    "",
    "3. **Output**",
    "   - final_answer: concise answer for the user",
    "   - can_be_answered: true/false",
]
JUDGE_PROMPT = "\n".join(JUDGE_PROMPT)

def main(*args, **kwargs):
    available_collections = [d.name for d in qdrant.get_collections().collections]
    print("Available collections in Qdrant:")
    for idx, name in enumerate(available_collections):
        print(f"{idx}: {name}")
    collection_index = int(input("Enter index of collection name to search: "))
    collection_name = available_collections[collection_index]
    while True:
        query = input("Enter your question: ")
        videos = search_videos(collection_name, query)

        for video in videos:
            print(f"Found video ID: {video.payload.get('video_id')} with score {video.score}, title: {video.payload.get('title')}")
        answers = []

        for video in videos:
            transcript = video.payload.get("transcript", "")
            video_id = video.payload.get("video_id", "unknown")

            if not transcript:
                print(f"⚠️ Skipping video {video_id}: no transcript available.")
                continue

            # Use system prompt + structured response
            structured_llm = llm.with_structured_output(QueryResponse)

            response = structured_llm.invoke(
                input=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Transcript:\n{transcript}\n\nQuestion: {query}"}
                ]
            )

            if response.can_be_answered:
                print(f"\n✅ Can be answered in video {video_id}:")
                print(f"Summary: {response.summary}")
                print(f"Time range: {response.start_time} - {response.end_time}")
                answers.append({
                    "video_id": video_id,
                    "title": video.payload.get("title"),
                    "upload_date": video.payload.get("upload_date"),
                    "score": video.score,
                    "response": response.model_dump()
                })
                # break
            else:
                print(f"\n❌ Cannot be answered in video {video_id}")
        judge_llm = llm.with_structured_output(FinalSummary)

        judge_input = {
        "role": "user",
        "content": f"User question: {query}\n\nVideo responses:\n" +
                    "\n\n".join([
                        f"Video: {a['title']} (ID: {a['video_id']}, Score: {a['score']:.3f})\n"
                        f"Video upload date: {a['upload_date']}\n"
                        f"Answer: {a['response']['summary']}\n"
                        f"Can be answered: {a['response']['can_be_answered']}\n"
                        f"Time range: {a['response']['start_time']} - {a['response']['end_time']}"
                        for a in answers
                    ])
        }

        final_result = judge_llm.invoke([
            {"role": "system", "content": JUDGE_PROMPT},
            judge_input
            ])

        if final_result.can_be_answered:
            print("\n✅ FINAL ANSWER:")
            print(f"Summary: {final_result.final_answer}")
            print(f"Sources: {', '.join(final_result.sources)}")
            print(f"Time ranges: {', '.join(final_result.time_ranges)}")
        else:
            print("\n❌ Could not confidently answer the question from available videos.")

# === Example usage ===
if __name__ == "__main__":
    main()


